In [2]:
!pip install tensorflow==2.3.0
!pip install tensorflow-addons==0.11.1
!pip install efficientnet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.1 MB 5.6 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.10.0
    Uninstalling tensorflow-addons-0.10.0:
      Successfully uninstalled tensorflow-addons-0.10.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 50 kB 4.7 MB/s eta 0:00:011
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

import efficientnet.tfkeras as efn #Convolutional Neural Network architecture
from sklearn.utils import class_weight as cw

In [4]:
tf.__version__

'2.3.0'

In [5]:
batch_size = 16
img_size = (256, 5150)
seed = 1
storage_dir = '../input/birdmel/train_img_split'
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

Found 50303 files belonging to 264 classes.
Using 45273 files for training.
Found 50303 files belonging to 264 classes.
Using 5030 files for validation.


In [7]:
class_num = len(train_ds.class_names)

In [8]:
train_datagen = ImageDataGenerator(dtype=np.float16)

train_generator = train_datagen.flow_from_directory(
    directory=storage_dir,
    class_mode="categorical",
    target_size=img_size,
    batch_size=batch_size
)

class_weights = cw.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes), 
    train_generator.classes
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}

del train_datagen
del train_generator

Found 50303 images belonging to 264 classes.


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 

In [9]:
# https://www.tensorflow.org/tutorials/images/data_augmentation#two_options_to_use_the_preprocessing_layers

random_crop_layer = tf.keras.layers.experimental.preprocessing.RandomCrop(256, 216)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)

def resize(image, height=256, width=256):
    return tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)

def augment_image_train(image):
    image = random_crop_layer(image)
    image = resize(image)
    image = normalization_layer(image)

    return image

def augment_image_test(image):
    image = resize(image, 256, 216)
    image = resize(image)
    image = normalization_layer(image)

    return image

In [10]:
def prepare(ds, augment=False):
    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (augment_image_train(x), y), num_parallel_calls=AUTOTUNE)
    else: 
        ds = ds.map(lambda x, y: (augment_image_test(x), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [11]:
train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)

In [17]:
# References:
# https://keras.io/api/applications/efficientnet/
# https://github.com/qubvel/efficientnet
net = efn.EfficientNetB5(
    include_top=False,
    weights='noisy-student',
    input_tensor=None,
    input_shape=(256, 256, 3)
)

x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(class_num, activation='softmax')(x)

# https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/
net_final = Model(inputs=net.input, outputs=output_layer)     
net_final.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy', 
    metrics=[tfa.metrics.F1Score(num_classes = class_num, average = 'micro')]
)

In [18]:
net_final.load_weights('./efficientnet_b5_checkpoint_tf_2_3_0.h5')

In [19]:
model_check = ModelCheckpoint(
    'efficientnet_b5_checkpoint_tf_2_3_0.h5', 
    monitor='val_f1_score', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True,
    mode='max',
    period=1,
)


reduce_LR = ReduceLROnPlateau(
    monitor='val_f1_score',
    factor=0.1,
    patience=5, 
    min_lr=1e-4
)

In [20]:
import gc
gc.collect()

45

In [ ]:
net_final.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 40,
    class_weight = class_weights,
    callbacks = [reduce_LR, model_check]
)

Epoch 1/40
2830/2830 [==============================] - 1044s 369ms/step - loss: 1.8905 - f1_score: 0.5321 - val_loss: 2.6779 - val_f1_score: 0.4809
Epoch 2/40
2830/2830 [==============================] - 1030s 364ms/step - loss: 1.9196 - f1_score: 0.5200 - val_loss: 2.6460 - val_f1_score: 0.4791
Epoch 3/40
2830/2830 [==============================] - 1038s 367ms/step - loss: 1.9350 - f1_score: 0.5225 - val_loss: 2.9010 - val_f1_score: 0.4549
Epoch 4/40
2117/2830 [=====================>........] - ETA: 3:55 - loss: 1.8868 - f1_score: 0.5257